## Set Up

In [ ]:
!pip install torch torchvision --quiet

In [ ]:
import time
import warnings
warnings.filterwarnings('ignore')

# Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Torch
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, models

# Keras
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications import ResNet50
import tensorflow.keras.layers as layers
import tensorflow as tf

# Evaluation
from sklearn.metrics import classification_report

# For reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Setting up device for GPU usage if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Running on {device}.')

# Data import
from tensorflow.keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

class_labels = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot')

Running on cpu.
4422102/4422102 [==============================] - 1s 0us/step


100%|██████████| 26421880/26421880 [00:00<00:00, 118000550.46it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 37298849.82it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 46438642.13it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 10267368.99it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



### Keras Model Parent Class

In [ ]:
class Keras_Neural_Net():

    model = None

    def compile(self, **kwargs):
        self.model.compile(**kwargs)

    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)

    def evaluate(self, *args, **kwargs):
        return self.model.evaluate(*args, **kwargs)

    def predict(self, *args, **kwargs):
        return self.model.predict(*args, **kwargs)

## Architecture 1: Basic Neural Net

In [ ]:
class Net1_Torch(nn.Module):

    def __init__(self):

        super(Net1_Torch, self).__init__()

        self.name = 'Net1_Torch'

        # self.rescale = nn.Softmax(dim=0)

        self.layer1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=128),
            nn.ReLU()
        )

        self.fc2 = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):

        # x = self.rescale(x)

        x = torch.div(x, torch.tensor(255))

        # Flattening
        x = x.view(x.size(0), -1)

        x = self.layer1(x)
        x = self.fc2(x)

        return x

In [ ]:
class Net1_Keras(Keras_Neural_Net):

    def __init__(self):

        self.name = 'Net1_Keras'

        self.model = Sequential([
            layers.Rescaling(scale=1./255),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(10)
        ])

## Architecture 2: CNN

In [ ]:
class Net2_Torch(nn.Module):

    def __init__(self):

        super(Net2_Torch, self).__init__()

        self.name = 'Net2_Torch'

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        self.fc3 = nn.Linear(in_features=64*5*5, out_features=128)
        self.fc4 = nn.Linear(in_features=128, out_features=len(class_labels))

    def forward(self, x):

        x = self.layer1(x)
        x = self.layer2(x)

        # Flattening
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc3(x))

        x = self.fc4(x)

        return F.log_softmax(x, dim=1)

In [ ]:
class Net2_Keras(Keras_Neural_Net):

    def __init__(self):

        self.name = 'Net2_Keras'

        self.model = Sequential([

            layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
            layers.MaxPooling2D(pool_size=(2, 2)),

            layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
            layers.MaxPooling2D(pool_size=(2, 2)),

            layers.Flatten(),

            layers.Dense(128, activation='relu'),

            layers.Dense(len(class_labels), activation='log_softmax')
        ])

## Architecture 3: CNN with Batch Normalization and Zero Padding

In [ ]:
class Net3_Torch(nn.Module):

    def __init__(self):

        super(Net3_Torch, self).__init__()

        self.name = 'Net3_Torch'

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=2)
        )

        self.fc4 = nn.Linear(in_features=256, out_features=1024)
        self.fc5 = nn.Linear(in_features=1024, out_features=512)
        self.fc6 = nn.Linear(in_features=512, out_features=len(class_labels))

    def forward(self, x):

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        # Flatten
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)

        return F.log_softmax(x, dim=1)

In [ ]:
class Net3_Keras(Keras_Neural_Net):

    def __init__(self):

        self.name = 'Net3_Keras'

        self.model = Sequential([

            layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
            layers.BatchNormalization(),
            layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D(pool_size=(2, 2)),

            layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D(pool_size=(2, 2)),

            layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.MaxPooling2D(pool_size=(2, 2)),

            layers.Flatten(),

            layers.Dense(1024, activation='relu'),

            layers.Dense(512, activation='relu'),

            layers.Dense(len(class_labels), activation='log_softmax')
        ])

## Architecture 4: CNN with Batch Normalization, Zero Padding and Dropout

In [ ]:
class Net4_Torch(nn.Module):

    def __init__(self):

        super(Net4_Torch, self).__init__()

        self.name = 'Net4_Torch'

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
        self.drop = nn.Dropout2d(p=0.25)
        self.fc2 = nn.Linear(in_features=600, out_features=120)
        self.fc3 = nn.Linear(in_features=120, out_features=10)

    def forward(self, x):

        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return F.log_softmax(x, dim=1)

In [ ]:
class Net4_Keras(Keras_Neural_Net):

    def __init__(self):

        self.name = 'Net4_Keras'

        self.model = Sequential([

            layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', input_shape=(28, 28, 1)),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling2D(pool_size=(2, 2)),

            layers.Conv2D(filters=64, kernel_size=(3, 3)),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.MaxPooling2D(pool_size=(2, 2)),

            layers.Flatten(),

            layers.Dense(600),
            layers.Dropout(rate=0.25),

            layers.Dense(120, activation='relu'),

            layers.Dense(len(class_labels), activation='log_softmax')
        ])

## Architecture 5: CNN with Residual Block

In [ ]:
class Net5_Torch(nn.Module):

    def __init__(self):

        super(Net5_Torch, self).__init__()

        self.name = 'Net5_Torch'

        # Load a pretrained resnet model from torchvision.models in Pytorch
        self.model = models.resnet50(pretrained=True)

        # Change the input layer to take Grayscale image, instead of RGB images.
        # Hence in_channels is set as 1 or 3 respectively
        # original definition of the first layer on the ResNet class
        self.model.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)

        # Change the output layer to output 10 classes instead of 1000 classes
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10)

    def forward(self, x):

        return self.model(x)

In [ ]:
class Net5_Keras(Keras_Neural_Net):

    def __init__(self):

        self.name = 'Net5_Keras'

        self.model = Sequential([

            # Load the pre-trained ResNet50 model without top layers (i.e., excluding the classification head)
            ResNet50(weights="imagenet", include_top=False, input_shape=(32, 32, 3)),

            layers.Flatten(),

            layers.Dense(1024, activation="relu"),

            layers.Dense(len(class_labels), activation="softmax")
        ])

    def resize(self, X):

        X = np.expand_dims(X, axis=-1)
        X = np.repeat(X, 3, axis=-1)
        X = X.astype('float32') / 255
        X = tf.image.resize(X, [32, 32])

        return X

    def fit(self, X, *args, **kwargs):

        return self.model.fit(self.resize(X), *args, **kwargs)

    def evaluate(self, X, *args, **kwargs):

        return self.model.evaluate(self.resize(X), *args, **kwargs)

    def predict(self, X, *args, **kwargs):

        return self.model.predict(self.resize(X), *args, **kwargs)

## Model Training

In [ ]:
def train_and_evaluate(model_class, num_epochs=5):

    if 'Net5' in model_class.name:
        num_epochs = 1

    if 'Torch' in model_class.name:
        return train_and_evaluate_torch(model_class, num_epochs)

    return train_and_evaluate_keras(model_class, num_epochs)

def train_and_evaluate_torch(m, num_epochs=5):

    model = m.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):

        print(f'Epoch {epoch + 1}/{num_epochs}')

        # Set the model to training mode
        model.train()

        start_time_sec = time.time()

        for i, (X_train, y_train) in enumerate(trainloader):

            # Training visualization
            progress = int((i + 1) / len(trainloader) * 30)
            progress_bar = '=' * (progress - 1) + '>' + '.' * (30 - progress)
            if (i + 1) == len(trainloader): progress_bar = '=' * 30
            elapsed = round(time.time() - start_time_sec, 2)
            print(f'\r{i + 1}/{len(trainloader)} [{progress_bar}] elapsed: {elapsed}s', end='', flush=True)

            # Move the images and labels to the computational device (CPU or GPU)
            X_train, y_train = X_train.to(device), y_train.to(device)

            # Clear the gradients from the previous iteration
            optimizer.zero_grad()

            # Forward pass: Pass the images through the model to get the predicted outputs
            y_pred = model(X_train)

            # Compute the cross entropy loss between the predicted outputs and the true labels
            loss = nn.CrossEntropyLoss()(y_pred, y_train)

            # Backward pass: Compute the gradient of the loss w.r.t. model parameters
            loss.backward()

            # Update the model parameters
            optimizer.step()

        print()

    model.eval()

    with torch.no_grad():

        correct = 0
        total = 0
        y_true = []
        y_pred = []

        for X_test, y_test in testloader:

            X_test, y_test = X_test.to(device), y_test.to(device)
            outputs = model(X_test)
            _, predicted = torch.max(outputs.data, 1)
            total += y_test.size(0)
            correct += (predicted == y_test).sum().item()
            predicted=predicted.to('cpu')
            y_test = y_test.to('cpu')
            y_true.extend(y_test)
            y_pred.extend(predicted)

        acc = correct / total

    report = classification_report(y_true, y_pred, target_names=class_labels)

    return m.name, acc, report

def train_and_evaluate_keras(model, num_epochs=5):

    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss=SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=num_epochs)

    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    y_pred = np.array([np.argmax(arr) for arr in model.predict(X_test, verbose=0)])
    report = classification_report(y_test, y_pred, target_names=class_labels)

    return model.name, acc, report

In [ ]:
model_classes = [Net1_Torch, Net1_Keras,
                 Net2_Torch, Net2_Keras,
                 Net3_Torch, Net3_Keras,
                 Net4_Torch, Net4_Keras,
                 Net5_Torch, Net5_Keras]

result = {
    'architecture': [],
    'accuracy': []
}

for model_class in model_classes:

    print('\n')

    arch, acc, _ = train_and_evaluate(model_class())
    result['architecture'].append(arch)
    result['accuracy'].append(acc)

Epoch 1/5
1875/1875 [==============================] elapsed: 48.83s
Epoch 2/5
1875/1875 [==============================] elapsed: 26.77s
Epoch 3/5
1875/1875 [==============================] elapsed: 26.61s
Epoch 4/5
1875/1875 [==============================] elapsed: 27.04s
Epoch 5/5
1875/1875 [==============================] elapsed: 26.64s
Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 0.5017 - accuracy: 0.8227
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3770 - accuracy: 0.8626
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3365 - accuracy: 0.8764
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3110 - accuracy: 0.8850
Epoch 5/5
313/313 [==============================] - 1s 2ms/step
Epoch 1/5
1875/1875 [==============================] elapsed: 106.19s
Epoch 2/5
1875/1875 [==============================] elapsed: 89.92s
Epoch 3/5
1875/1875 [=======================

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 107MB/s]


Epoch 1/1
313/313 [==============================] - 52s 156ms/step


In [ ]:
pd.DataFrame(result).sort_values(by='accuracy', ascending=False)

,architecture,accuracy
5,Net3_Keras,0.9249
4,Net3_Torch,0.9134
7,Net4_Keras,0.9125
6,Net4_Torch,0.9123
2,Net2_Torch,0.8997
3,Net2_Keras,0.8936
1,Net1_Keras,0.8715
8,Net5_Torch,0.8120
9,Net5_Keras,0.8110
0,Net1_Torch,0.8002
